In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("/Users/starboy/Documents/Projects/credit_risk_segmentation/data/processed/model_ready_data.csv")

##### **DEFAULT RATE BY FEATURE BUCKETS** 

In [4]:
df["loan_to_income_buckets"] = pd.cut(
    df["loan_to_income"],
    bins=[0,0.1,0.25,1],
    labels = ["Low","Medium","High"]
)

In [5]:
df.groupby("loan_to_income_buckets")["target"].mean()

/var/folders/_w/n63k25_948b11n_ktsfq3_dh0000gn/T/ipykernel_908/3137142877.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("loan_to_income_buckets")["target"].mean()


loan_to_income_buckets
Low       0.018692
Medium    0.016304
High      0.012579
Name: target, dtype: float64

The default rates are going down. This dataset has people with higher loan_to_income defaulting less.

In [6]:
df["loan_to_income_buckets"].value_counts()

loan_to_income_buckets
Medium    184
High      159
Low       107
Name: count, dtype: int64

In [11]:
df["dti_bucket"] = pd.cut(
    df["debt_to_income"],
    bins=[-1,20,35,100],
    labels=["Low","Medium","High"]
)

In [12]:
df.groupby("dti_bucket")["target"].mean()


/var/folders/_w/n63k25_948b11n_ktsfq3_dh0000gn/T/ipykernel_908/2475606493.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("dti_bucket")["target"].mean()


dti_bucket
Low       0.010135
Medium    0.028369
High      0.000000
Name: target, dtype: float64

In [13]:
df["dti_bucket"].value_counts()


dti_bucket
Low       296
Medium    141
High       16
Name: count, dtype: int64

because of approval bias, banks dont approve loans for people with very high debt. so most high-DTI risky people were never approved and they never entered the dataset.

SEGMENTATION

1. segment 1 - Low stress (low loan_to_income; low debt_to_income)
2. segment 2 - Loan Heavy (high loan_to_income ; low debt_to_income)
3. segment 3 - Debt Stressed (medium loan_to_income ; medium debt_to_income)
4. segment 4 - Thin Credit (credit_history_years < 5>)

In [14]:
def assign_segment(row):
    if row["credit_history_years"] < 5 :
        return "Thin Credit"
    if row["loan_to_income"] > 0.25 and row["debt_to_income"]<20:
        return "Loan Heavy"
    if row["debt_to_income"] >= 20:
        return "Debt Stressed"
    return "Low Stress"

In [15]:
df["customer_segment"] = df.apply(assign_segment, axis=1)

In [16]:
df["customer_segment"].value_counts()

customer_segment
Low Stress       200
Debt Stressed    156
Loan Heavy        91
Thin Credit        7
Name: count, dtype: int64

##### **CUSTOMER SEGMENT**

1. **Low Stress** - small loans, low existing debt
2. **Debt Stressed** - already paying a lot of old loans
3. **Loan Heavy** - big new loan, but not much old debt
4. **Thin Credit** - very little credit history

In [17]:
df.groupby("customer_segment")["target"].mean()

customer_segment
Debt Stressed    0.025641
Loan Heavy       0.000000
Low Stress       0.015000
Thin Credit      0.000000
Name: target, dtype: float64

out of n people like this, how many failed to repay ? - Percentages comes in. 

1. Debt Stressed ~ 2.5% - out of 100 people who already had a lot of debt, about 2-3 people failed. So this group is riskier than others.
2. Low Stress ~ 1.5% - people who were allowed to take big loans were usually strong and repaid
3. Thin Credit 0% - Only 7 people are here we dont have enough people to say anything. 
